In [15]:
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import os
import pickle
from lifelines.utils import concordance_index

from utils import dataset_name
from eval_utils import MODEL_CLASS, get_results, DL_MODELS, ML_MODELS, MODELS
import numpy as np
import pandas as pd
np.random.seed(42)
_ = torch.manual_seed(42)


print(f"Experiments over {dataset_name}")

Experiments over SUPPORT


In [16]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

# Finding the best parameters for each model and run the models over the test data

In [17]:
results = pd.DataFrame()
     
for model in MODELS:
    stats = pickle.load( open( f"statistics/{model}/best_model.pkl", "rb" ))
    df = pd.DataFrame(stats, index=[model])
    results = pd.concat([results, df], axis=0)
    
c_index_df = results[["c_index", "c_index_std", "c_index_params"]].sort_values("c_index", ascending=False)
concordance_td_df = results[["concordance_td", "concordance_td_std", "concordance_td_params"]].sort_values("concordance_td", ascending=False)
ibs_df = results[["ibs", "ibs_std", "ibs_params"]].sort_values("ibs", ascending=True)

## `c index`

In [18]:
c_index_df

,c_index,c_index_std,c_index_params
gbst,0.613528,0.008532,gbst_learning_rate_0.1_n_estimators_100_subsam...
deep_surv,0.611813,0.011053,"deep_surv_dropout_0.2_num_nodes_[32, 32]_activ..."
rsf,0.610467,0.010598,rsf_n_estimators_200_max_depth_10_min_samples_...
cox_time,0.573897,0.011235,"cox_time_dropout_0.1_num_nodes_[32, 32]_activa..."
reg_coxph,0.568514,0.013525,reg_coxph_l1_ratio_0.1_tol_1e-07_max_iter_1000...
pc_hazard,0.527059,0.021539,"pc_hazard_dropout_0.1_num_nodes_[16, 16, 16, 1..."
deep_hit,0.495508,0.009164,"deep_hit_dropout_0.3_num_nodes_[3, 5]_activati..."


In [19]:
results = []
column = "c_index_params"


for model_name, params in zip(c_index_df.index, c_index_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, c_index))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

gbst 0.6283
deep_surv 0.6276
rsf 0.6247
cox_time 0.6098
reg_coxph 0.5902
deep_hit 0.5173
pc_hazard 0.4981


___

## `concordance td`

In [20]:
concordance_td_df

,concordance_td,concordance_td_std,concordance_td_params
rsf,0.628067,0.005299,rsf_n_estimators_200_max_depth_5_min_samples_s...
deep_hit,0.624383,0.010346,"deep_hit_dropout_0.1_num_nodes_[28, 28, 100, 2..."
gbst,0.613528,0.008532,gbst_learning_rate_0.1_n_estimators_100_subsam...
deep_surv,0.611813,0.011053,"deep_surv_dropout_0.2_num_nodes_[32, 32]_activ..."
cox_time,0.590286,0.013399,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
reg_coxph,0.568514,0.013525,reg_coxph_l1_ratio_0.1_tol_1e-07_max_iter_1000...
pc_hazard,0.222859,0.006155,pc_hazard_dropout_0.1_num_nodes_[100]_activati...


In [21]:
results = []
column = "concordance_td_params"


for model_name, params in zip(concordance_td_df.index, concordance_td_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, concordance_td))

for m,val in sorted(results, key=lambda x: x[1], reverse=True):
    print(m, np.round(val,4))

rsf 0.6428
gbst 0.6283
deep_surv 0.6276
deep_hit 0.6195
cox_time 0.6159
reg_coxph 0.5902
pc_hazard 0.4731


## `integrated brier score`

In [22]:
ibs_df

,ibs,ibs_std,ibs_params
pc_hazard,0.023602,0.000701,"pc_hazard_dropout_0.3_num_nodes_[16, 16, 16, 1..."
deep_hit,0.066380,0.000796,"deep_hit_dropout_0.1_num_nodes_[32, 32]_activa..."
deep_surv,0.191749,0.002846,"deep_surv_dropout_0.2_num_nodes_[32, 32]_activ..."
rsf,0.192619,0.003168,rsf_n_estimators_200_max_depth_10_min_samples_...
gbst,0.192729,0.003622,gbst_learning_rate_0.1_n_estimators_100_subsam...
cox_time,0.205687,0.004516,"cox_time_dropout_0.1_num_nodes_[32, 128, 128]_..."
reg_coxph,0.206085,0.003630,reg_coxph_l1_ratio_1.0_tol_1e-09_max_iter_1000...


In [23]:
results = []
column = "ibs_params"


for model_name, params in zip(ibs_df.index, ibs_df[column]):
    if model_name in DL_MODELS:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))
    else:
        c_index, concordance_td, ibs = get_results(params, model_name, X, y, X_test, y_test)
        results.append((model_name, ibs))

for m,val in sorted(results, key=lambda x: x[1], reverse=False):
    print(m, np.round(val,4))

deep_surv 0.1806
gbst 0.181
rsf 0.1816
cox_time 0.1917
reg_coxph 0.1954
deep_hit 0.2856
pc_hazard 0.3248
